In [3]:
import pandas as pd
from pyspark import SparkContext
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import json

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
# Attempt at extracting local JSON file without using pyspark module 

# pyspark module seems un-nessisary for this part, plus using SQL queries for
# transformations would be a huge pain in the ass. 

sc = SparkContext(appName="Spark-Test")


In [5]:
sc

<SparkContext master=local[*] appName=Spark-Test>

In [10]:
rdd_customer = sc.textFile("../data/credit_card_dataset/cdw_sapp_custmer.json")

In [17]:
# type of return with SparkContext.textFile method
print(type(rdd_customer)) 
 

zeroth_item = rdd_customer.take(1)
# contents of take.(1)
print(zeroth_item)

# take produces list
print(type(zeroth_item))

# returns string
print(type(zeroth_item[0]))


<class 'pyspark.rdd.RDD'>
['{"FIRST_NAME":"Alec","MIDDLE_NAME":"Wm","LAST_NAME":"Hooper","SSN":123456100,"CREDIT_CARD_NO":"4210653310061055","APT_NO":"656","STREET_NAME":"Main Street North","CUST_CITY":"Natchez","CUST_STATE":"MS","CUST_COUNTRY":"United States","CUST_ZIP":"39120","CUST_PHONE":1237818,"CUST_EMAIL":"AHooper@example.com","LAST_UPDATED":"2018-04-21T12:49:02.000-04:00"}']
<class 'list'>
<class 'str'>


In [18]:
rdd_customer_json = rdd_customer.map(lambda json_item: json.loads(json_item) )  # similar to apply in pandas or just regular map func in python

In [26]:
print(type(rdd_customer_json), "\n")

zeroth_item_json = rdd_customer_json.take(1)
print(zeroth_item_json, "\n")

print(type(zeroth_item_json[0]["CUST_ZIP"]))  # converted to json-like dict but all values are still strings


<class 'pyspark.rdd.PipelinedRDD'> 

[{'FIRST_NAME': 'Alec', 'MIDDLE_NAME': 'Wm', 'LAST_NAME': 'Hooper', 'SSN': 123456100, 'CREDIT_CARD_NO': '4210653310061055', 'APT_NO': '656', 'STREET_NAME': 'Main Street North', 'CUST_CITY': 'Natchez', 'CUST_STATE': 'MS', 'CUST_COUNTRY': 'United States', 'CUST_ZIP': '39120', 'CUST_PHONE': 1237818, 'CUST_EMAIL': 'AHooper@example.com', 'LAST_UPDATED': '2018-04-21T12:49:02.000-04:00'}] 

<class 'str'>


In [30]:
# Attempt at using local json file as basis for df with pyspark.sql
pyspark = SparkSession.builder.appName("Spark-SQL-Test").getOrCreate()


In [31]:
customer_df = pyspark.read.json("../data/credit_card_dataset/cdw_sapp_custmer.json")

In [35]:
print(type(zeroth_item_json[0]["CUST_PHONE"]))

<class 'int'>


In [34]:
customer_df.printSchema() 
# both vanilla pyspark and pyspark.sql produce basically the same results,
# with the same default types

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [36]:
rdd_df = rdd_customer_json.toDF()

In [37]:
rdd_df.show()  # rdd translates into a df nicely

+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States| WDunham@exam